# Analyze histogram of predicted scores

In [ ]:
import sys
sys.path.append("./../")

In [ ]:
import os
import json
import glob

import numpy as np
from scipy import stats as S
from tbparse import SummaryReader

import methods, models, datasets, transforms

import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [ ]:
def get_event_file(model_dir):
    tblog_dir = os.path.join(model_dir, "tblog")
    events_file = glob.glob(tblog_dir + "/events.out.tfevents*.1")[0] # *.1 will be from test
#     events_file = glob.glob(tblog_dir + "/events.out.tfevents*.0")[0] # *.1 will be from test

    return events_file

In [ ]:
model_dir = "./../zoo/experiments-termwise-ablation/BinaryMNISTC-53-identity/LeNet/term_1_3-20220402202026"
# model_dir = "./../zoo/experiments-termwise-ablation/BinaryMNISTC-53-identity/LeNet/term-all-beta0101-20220402214708"
# model_dir = "./../zoo/experiments-termwise-ablation/BinaryMNISTC-53-identity/LeNet/term-all-beta1010-20220402213949"
# model_dir = "./../zoo/experiments-termwise-ablation/BinaryMNISTC-53-identity/LeNet/term-all-beta5050-20220402213330"
# model_dir = "./../zoo/experiments-termwise-ablation/BinaryMNISTC-53-identity/LeNet/abl-beta0101-lamsl-1e-3-20220405095409"
# model_dir = "./../zoo/experiments-termwise-ablation/BinaryMNISTC-53-identity/LeNet/abl-beta0101-lamsl-1e-2-20220405101410"
# model_dir = "./../zoo/experiments-termwise-ablation/BinaryMNISTC-53-identity/LeNet/abl-beta0101-lamsl-1e-1-20220405102339/"
# model_dir = "./../zoo/experiments-termwise-ablation/BinaryMNISTC-53-identity/LeNet/abl-beta1010-lamsl-1e-3-20220405103601/"
# model_dir = "./../zoo/experiments-termwise-ablation/BinaryMNISTC-53-identity/LeNet/abl-beta1010-lamsl-1e-2-20220405104817/"
# model_dir = "./../zoo/experiments-termwise-ablation/BinaryMNISTC-53-identity/LeNet/abl-beta1010-lamsl-1e-1-20220405105703/"
# model_dir = "./../zoo/experiments-termwise-ablation/BinaryMNISTC-53-identity/LeNet/abl-beta5050-lamsl-1e-3-20220405110933/"
# model_dir = "./../zoo/experiments-termwise-ablation/BinaryMNISTC-53-identity/LeNet/abl-beta5050-lamsl-1e-2-20220405112231/"
# model_dir = "./../zoo/experiments-termwise-ablation/BinaryMNISTC-53-identity/LeNet/abl-beta5050-lamsl-1e-1-20220405113534/"
# model_dir = "./../zoo/experiments-termwise-ablation/BinaryMNISTC-53-identity/LeNet/dir-5050-lamsl-1e0-20220513202814/"

# Default paths
config_json = os.path.join(model_dir, "config.json")
ckpt_file = os.path.join(model_dir, "last.ckpt")
event_file = get_event_file(model_dir)


In [ ]:
config = json.load(open(config_json, 'r'))
params = config['method_params']

ev = SummaryReader(event_file, pivot=True)
df_scalars = ev.scalars
df_hist = ev.histograms

In [ ]:
df_scalars

In [ ]:
l = df_hist.iloc[-1]
x = l['test_y_pred/limits']
y = l['test_y_pred/counts']
# x = l['y_pred/limits']
# y = l['y_pred/counts']
y = y / y.sum()


base_measure = S.beta(params['a'], params['b'])
x2 = np.linspace(0.0, 1.0, 25)
pdf = base_measure.pdf(x2)


fig = plt.figure(figsize=(6,4))
# plt.fill_between(x, y, step='pre')
plt.bar(x, y, width=-0.09, align='edge')
# plt.plot(x2, pdf)
plt.xlim(0.0, 1.0)
plt.ylim(0.0, 0.6)
# plt.title(
#     "Beta({:.2f}, {:.2f})".format(params['a'], params['b']) +
#     "    $\\alpha$ = {:.0f}".format(params['alpha']) +
#     "    $\\lambda_{SL}$ = "+"{:.2f}".format(params['lam_sl'])
# )
plt.xlabel("$\\hat{p}$")
plt.ylabel("$\hat{s_{\\theta}}$")
# plt.tight_layout()